# Ellipsometry workflow example

## Create NeXus file from measurement data

In [1]:
from nexusparser.tools.dataconverter.convert import convert

In [2]:
convert(input_file=["test.yaml"],
        reader='ellips',
        nxdl='NXellipsometry',
        output='ellips.test.nxs')

Using ellips reader to convert the given files:  
• test.yaml 
The path, /ENTRY[entry]/plot/wavelength, is being written but has no documentation.
The path, /ENTRY[entry]/plot/wavelength/@units, is being written but has no documentation.
The path, /ENTRY[entry]/plot/psi_50deg, is being written but has no documentation.
The path, /ENTRY[entry]/plot/psi_50deg/@units, is being written but has no documentation.
The path, /ENTRY[entry]/plot/psi_60deg, is being written but has no documentation.
The path, /ENTRY[entry]/plot/psi_60deg/@units, is being written but has no documentation.
The path, /ENTRY[entry]/plot/psi_70deg, is being written but has no documentation.
The path, /ENTRY[entry]/plot/psi_70deg/@units, is being written but has no documentation.
The path, /ENTRY[entry]/plot/delta_50deg, is being written but has no documentation.
The path, /ENTRY[entry]/plot/delta_50deg/@units, is being written but has no documentation.
The path, /ENTRY[entry]/plot/delta_60deg, is being written but has

## Inspect the NeXus file with h5web

In [3]:
from jupyterlab_h5web import H5Web

In [4]:
H5Web('ellips.test.nxs')

<jupyterlab_h5web.widget.H5Web object>

Here is where the general template ends. Continue to fill the notebook based on<br>
**your own** post-processing of the *.nxs file.

## Analyze $\Psi$ \ $\Delta$ values with a transfer-matrix solver

In [1]:
import elli
from elli.fitting import ParamsHist, fit
from elli.nexus import read_psi_delta

### Load data from NeXus file

In [2]:
psi_delta = read_psi_delta('ellips.test.nxs').loc[210:800]

### Set model parameters

In [3]:
params = ParamsHist()
params.add("SiO2_n0", value=1.452, min=-100, max=100, vary=True)
params.add("SiO2_n1", value=36.0, min=-40000, max=40000, vary=True)
params.add("SiO2_n2", value=0, min=-40000, max=40000, vary=True)
params.add("SiO2_k0", value=0, min=-100, max=100, vary=True)
params.add("SiO2_k1", value=0, min=-40000, max=40000, vary=True)
params.add("SiO2_k2", value=0, min=-40000, max=40000, vary=True)
params.add("SiO2_d", value=2, min=0, max=40000, vary=True)

### Build the model and show interactive plot to set the parameters

In [4]:
@fit(psi_delta, params)
def model(lbda, params):
    sr = elli.SpectraRay("./")
    Si = elli.IsotropicMaterial(sr.loadDispersionTable("Si_Aspnes.mat"))

    SiO2 = elli.Cauchy(
        params["SiO2_n0"],
        params["SiO2_n1"],
        params["SiO2_n2"],
        params["SiO2_k0"],
        params["SiO2_k1"],
        params["SiO2_k2"],
    ).get_mat()

    Layer = [elli.Layer(SiO2, params["SiO2_d"])]

    return elli.Structure(elli.AIR, Layer, Si).evaluate(lbda, 50, solver=elli.Solver2x2)

### Fit and plot fit result

In [5]:
fit_stats = model.fit()
model.plot()

FigureWidget({
    'data': [{'hovertemplate': 'variable=Ψ<br>Wavelength=%{x}<br>value=%{y}<extra></extra>',
  …

## Show fit statistics

In [6]:
fit_stats